# Single-layer Neural Nets

In [1]:
import torch
import torch.nn as nn

In [ ]:
class MyModel(nn.Module):
    
    def __init__(self, in_dim=300, out_dim=100):
        super().__init__()
        self.wxb = nn.Linear(in_features=in_dim, out_features=out_dim, bias=True)
        
    
    def forward(self, input):
        out = self.wxb(input)
        return out


In [83]:
mdl = MyModel(in_dim=300, out_dim=100)
audio = torch.randn((3, 300))
out = mdl(audio)
print(out.shape)

torch.Size([3, 100])


# Multi layer Neural Nets

In [16]:
class MultiMyModel(nn.Module):

    def __init__(self, in_dim=300, hidden_dim=200, out_dim=100):
        super().__init__()
        self.nn1 = nn.Linear(in_features=in_dim, out_features=hidden_dim)
        self.ac1 = nn.ReLU()
        self.nn2 = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
        self.ac2 = nn.Sigmoid()
        self.nn3 = nn.Linear(in_features=hidden_dim, out_features=out_dim)

    def forward(self, input):
        nn1out = self.nn1(input)
        ann1out = self.ac1(nn1out)
        nn2out = self.nn2(ann1out)
        ann2out = self.ac2(nn2out)
        nn3out = self.nn3(ann2out)
        return nn3out

In [82]:
mymultlayer = MultiMyModel(in_dim=300, hidden_dim=200, out_dim=3)
audio = torch.randn((4, 300))
out = mymultlayer(audio)
print(out.shape)

torch.Size([4, 3])


# Loss

In [101]:
# Binary cross entropy
audio = torch.randn((4, 300))
single_layer = MyModel(in_dim=300, out_dim=1)
# don't do softmax in the model, because the loss functions expects 'logits'
# logits means the output before applying the softmax
out = single_layer(audio)
label = torch.Tensor([[0, 0, 1, 1]])
label = label.T
print(label.shape)

loss_fn = nn.BCEWithLogitsLoss()
# https://docs.pytorch.org/docs/stable/nn.html#loss-functions
loss = loss_fn(out, label)
print(loss)



torch.Size([4, 1])
tensor(0.5243, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [98]:
# cross entropy loss
audio = torch.randn((4, 300))
multi_layer = MultiMyModel(in_dim=300, out_dim=3)
# also don't apply softmax, based on cross entropy's low level 
# implementation, it also requires logits as inputs
# link: https://github.com/pytorch/pytorch/blob/d38164a545b4a4e4e0cf73ce67173f70574890b6/torch/nn/modules/loss.py#L1194
out = multi_layer(audio)
label = torch.tensor([0, 1, 2, 1], dtype=int)
print(label.shape)

ce_loss_fn = nn.CrossEntropyLoss()
ce_loss = ce_loss_fn(out, label)
print(ce_loss)

torch.Size([4])
tensor(1.2139, grad_fn=<NllLossBackward0>)


In [93]:
# more functions can be found
# https://docs.pytorch.org/docs/stable/nn.html#loss-functions

# How to do backward?

In [102]:
# if we don't do backward, no graidents would be calculated
for name, param in single_layer.named_parameters():
    print(name, param.shape, param.grad)

wxb.weight torch.Size([1, 300]) None
wxb.bias torch.Size([1]) None


In [99]:
# if we do backward, all parameter tensors would now have their gradients
ce_loss.backward()
for name, param in multi_layer.named_parameters():
    print(name, param.shape, param.grad)

nn1.weight torch.Size([200, 300]) tensor([[ 2.6152e-04, -1.8416e-04, -9.4570e-05,  ...,  6.1853e-04,
          1.5009e-04, -2.5850e-04],
        [-8.6773e-05,  4.4729e-05, -2.2361e-04,  ...,  9.8543e-05,
         -1.5698e-05, -7.5120e-05],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [-9.3883e-04, -1.0768e-04, -2.0826e-03,  ...,  1.1355e-03,
          3.6906e-04, -9.3242e-04],
        [-6.3052e-05, -2.8858e-04, -5.9982e-05,  ...,  2.5686e-04,
         -1.2360e-05, -1.6282e-04],
        [-3.7738e-04,  5.9500e-04, -9.2371e-04,  ..., -2.2870e-06,
         -9.0266e-05, -8.6243e-05]])
nn1.bias torch.Size([200]) tensor([ 5.8621e-04,  1.2575e-04,  0.0000e+00, -8.6024e-04,  1.2154e-04,
        -1.4094e-03, -1.4220e-03,  1.2874e-03, -2.5511e-04,  2.7287e-04,
        -9.3017e-04, -1.1675e-03, -8.8709e-04, -1.5700e-03, -3.9395e-04,
         1.7214e-03, -2.8490e-03, -2.9439e-04,  9.5461e-04, -1.7642e-03,
         3.9491